#  <u> 1st step : Load

Open the final merge table of the 'extraction part '

In [5]:
# we import the useful librairies 

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import functools
import os
import plotly.express as px
from scipy.stats import shapiro
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual


# we open and read the merge table of all indicators
dataset = pd.read_csv ('all_indicators_table.csv')
dataset

,Unnamed: 0,Entity,Code,Year,Deaths,LifeExpectancy,GenderInequality,GDP,Fertility,tertiary_education
0,0,Afghanistan,AFG,2005,118653,58.4,0.748,3.451279e+09,6.9047,3.61
1,1,Afghanistan,AFG,2010,101658,60.9,0.753,1.031396e+10,6.0987,3.65
2,2,Albania,ALB,2000,1449,75.4,0.319,2.400530e+09,2.2310,3.51
3,3,Albania,ALB,2005,815,76.6,0.306,4.487185e+09,1.8026,2.35
4,4,Albania,ALB,2010,473,77.9,0.246,7.718874e+09,1.6564,0.93
...,...,...,...,...,...,...,...,...,...,...
637,637,Zimbabwe,ZWE,1990,29244,59.4,0.605,4.946844e+09,4.8733,1.83
638,638,Zimbabwe,ZWE,1995,35883,50.5,0.594,4.506185e+09,4.1539,1.32
639,639,Zimbabwe,ZWE,2000,35389,44.7,0.625,4.421600e+09,3.9740,0.53
640,640,Zimbabwe,ZWE,2005,36797,44.8,0.595,3.165357e+09,3.6651,0.44


#  <u> 2nd step : Remove columns
    
Remove the column country names, not useful because the column code provides the country code

In [22]:
dataset_small = dataset.drop(dataset.columns[0], axis = 1)
dataset_small = dataset_small.drop(dataset_small.columns[0], axis = 1)
dataset_small

,Code,Year,Deaths,LifeExpectancy,GenderInequality,GDP,Fertility,tertiary_education
0,AFG,2005,118653,58.4,0.748,3.451279e+09,6.9047,3.61
1,AFG,2010,101658,60.9,0.753,1.031396e+10,6.0987,3.65
2,ALB,2000,1449,75.4,0.319,2.400530e+09,2.2310,3.51
3,ALB,2005,815,76.6,0.306,4.487185e+09,1.8026,2.35
4,ALB,2010,473,77.9,0.246,7.718874e+09,1.6564,0.93
...,...,...,...,...,...,...,...,...
637,ZWE,1990,29244,59.4,0.605,4.946844e+09,4.8733,1.83
638,ZWE,1995,35883,50.5,0.594,4.506185e+09,4.1539,1.32
639,ZWE,2000,35389,44.7,0.625,4.421600e+09,3.9740,0.53
640,ZWE,2005,36797,44.8,0.595,3.165357e+09,3.6651,0.44


#  <u> 3rd step : INTEGRATION,
I start filter the datas, first by country so I made groups of similar countries
    
I choose to make some groups according to their income. I made 4 groups : 

    - Low-income economies (1,135 dollars or less)  

    - Lower-middle-income economies (1,136 to 4,465 dollars)    

    - Upper-middle-income economies (4,466 to  13,845 dollars) 

    - High-income economies (13,845 dollars or more)  

In [23]:
low_income = ['AFG', 'BDI', 'CAF', 'COG', 'GMB', 'LBR', 'MWI', 'MLI', 'MOZ', 'NER', 'RWA', 'SLE', 'SDN', 'TGO', 'UGA', 'YEM'] 
lower_middle_income = ['DZA', 'BGD', 'BOL', 'KHM', 'CMR', 'COD', 'CIV', 'EGY', 'SWZ', 'GHA', 'HTI', 'HND', 'IDN', 'JOR', 'IRN', 'KEN', 'KGZ', 'LSO', 'MRT', 'MNG', 'MAR', 'NPL', 'NIC', 'NER', 'PAK', 'PNG', 'PHL', 'SEN', 'LKA', 'TJK', 'TZA', 'TUN', 'UKR', 'VNM', 'ZMB', 'ZWE']  
upper_middle_income = ['ALB', 'ARG', 'ARM', 'BLZ', 'BWA', 'BRA', 'BGR', 'CHN', 'COL', 'CRI', 'CUB', 'DOM', 'ECU', 'SLV', 'FJI', 'GAB', 'GTM', 'IDN', 'IRQ', 'JAM', 'KAZ', 'LBY', 'MYS', 'MDV', 'MUS', 'MEX', 'MDA', 'NAM', 'PRY', 'PER', 'ZAF', 'THA', 'TON'] 
high_income = ['AUS', 'AUT', 'BRB', 'BEL', 'CAN', 'CHL', 'HRV', 'CYP', 'DNK', 'EST', 'FIN', 'FRA', 'GUY', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ISR', 'ITA', 'JPN', 'KWT', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'NZL', 'NOR', 'PAN', 'POL', 'PRT', 'QAT', 'ROU', 'SGP', 'ESP', 'SWE', 'CHE', 'TTO', 'ARE', 'GBR', 'USA', 'URY'] 

country_list=low_income+lower_middle_income+upper_middle_income+high_income

We reduce the dataset with only groups of levels of incomes, we can also filter by indicators or even by year if we reduce the min and max dates

However, our dataset is not so big so maybe we will first work on the merge dataset to have more datas before use the filtered dataset

In [24]:
dataset_countries=dataset_small.loc[dataset_small['Code'].isin(country_list)]
dataset_countries

,Code,Year,Deaths,LifeExpectancy,GenderInequality,GDP,Fertility,tertiary_education
0,AFG,2005,118653,58.4,0.748,3.451279e+09,6.9047,3.61
1,AFG,2010,101658,60.9,0.753,1.031396e+10,6.0987,3.65
2,ALB,2000,1449,75.4,0.319,2.400530e+09,2.2310,3.51
3,ALB,2005,815,76.6,0.306,4.487185e+09,1.8026,2.35
4,ALB,2010,473,77.9,0.246,7.718874e+09,1.6564,0.93
...,...,...,...,...,...,...,...,...
637,ZWE,1990,29244,59.4,0.605,4.946844e+09,4.8733,1.83
638,ZWE,1995,35883,50.5,0.594,4.506185e+09,4.1539,1.32
639,ZWE,2000,35389,44.7,0.625,4.421600e+09,3.9740,0.53
640,ZWE,2005,36797,44.8,0.595,3.165357e+09,3.6651,0.44


Then we put all the indicators in only one column 

In [26]:
bronze_dataset=(dataset_countries.set_index(["Code", "Year"]).stack().reset_index(name='Value').rename(columns={'level_2':'Indicator'}))
bronze_dataset

,Code,Year,Indicator,Value
0,AFG,2005,Deaths,1.186530e+05
1,AFG,2005,LifeExpectancy,5.840000e+01
2,AFG,2005,GenderInequality,7.480000e-01
3,AFG,2005,GDP,3.451279e+09
4,AFG,2005,Fertility,6.904700e+00
...,...,...,...,...
3517,ZWE,2010,LifeExpectancy,5.070000e+01
3518,ZWE,2010,GenderInequality,5.920000e-01
3519,ZWE,2010,GDP,6.097825e+09
3520,ZWE,2010,Fertility,4.024800e+00


In [27]:
bronze_dataset.to_csv('bronze_dataset.csv')